# Scrape A Band's Wikipedia Member Timeline

In [3]:
import requests
import wikipedia
from selectolax.parser import HTMLParser

In [128]:
url = 'https://en.wikipedia.org/w/index.php?title=Arctic_Monkeys&action=edit'
url = 'https://en.wikipedia.org/w/index.php?title=Nirvana_(band)&action=edit'

In [129]:
# wikipedia.page('Nirvana (band)').section('#Timeline')
# url = wikipedia.page('Nirvana (band)').url
print('Url is', url)
resp = requests.get(url)
html = resp.content
tree = HTMLParser(html)

Url is https://en.wikipedia.org/w/index.php?title=Nirvana_(band)&action=edit


In [131]:
import requests
from bs4 import BeautifulSoup
import re

# send a GET request to the webpage
response = requests.get(url)

# parse the HTML content of the webpage
soup = BeautifulSoup(response.content, 'html.parser')

# find the textarea element
textarea = soup.find('textarea')

# extract the content of the textarea
textarea_content = textarea.get_text()

# process the extracted text as required
# print(textarea_content)

# use regular expressions to extract the timeline section
# match = re.search(r"{{#tag:timeline(.*?)}}", textarea_content, flags=re.DOTALL)
pattern = r"{{#tag:timeline\|([\s\S]*?)(?=(?<!{{).(?:{{(?!.*}})|}}))"
match = re.search(pattern, textarea_content, flags=re.DOTALL)
if match:
    timeline_text = match.group(1)
    print(timeline_text)

if match:
    timeline_section = match.group(1)
else:
    timeline_section = ''

# print the timeline section
print(timeline_section)


ImageSize = width:960 height:auto barincrement:22
PlotArea = left:90 bottom:90 top:0 right:0
Alignbars = justify
DateFormat = dd/mm/yyyy
Period = from:01/03/1987 till:05/04/1994
TimeAxis = orientation:horizontal format:yyyy
Legend = orientation:vertical position:bottom columns:4
ScaleMajor = increment:1 start:1988

Colors =
  id:V      value:red             legend:Lead_vocals
  id:BV     value:pink            legend:Backing_vocals
  id:LG     value:teal            legend:Lead_guitar
  id:RG     value:brightgreen     legend:Rhythm_guitar
  id:B      value:blue            legend:Bass
  id:D      value:orange          legend:Drums
  id:C      value:purple          legend:Cello
  id:T      value:yellow          legend:Touring_member
  id:studio value:black           legend:Studio_releases
  id:other  value:gray(0.5)       legend:Other_releases
  id:bars   value:gray(0.93)

BackgroundColors = bars:bars

LineData =
 layer:back color:studio
  at:15/06/1989
  at:24/09/1991
  at:13/09/1993
 la

In [161]:
bar_matches

[]

In [156]:
# define the regular expressions to extract BarData and PlotData
bar_regex = r"bar:(\w+)\s+text:(\w+\s*\w+)"
plot_regex = r"bar:(\w+)\s+from:([\w/]+)\s+till:([\w/]+)\s+color:(\w+)"

# the text containing the data

# find the BarData and store in a dictionary
bar_data = {}
bar_matches = re.findall(bar_regex, timeline_section)
for match in bar_matches:
    bar_data[match[0]] = match[1]

# find the PlotData and store in a dictionary
plot_data = {}
plot_matches = re.findall(plot_regex, timeline_section)
for match in plot_matches:
    plot_data[match[0]] = {
        "start": match[1],
        "end": match[2],
        "color": match[3]
    }

print("BarData:", bar_data)
print("PlotData:", plot_data)

BarData: {}
PlotData: {'KC': {'start': '19/07/1989', 'end': '23/09/1993', 'color': 'RG'}, 'KN': {'start': '30/06/1992', 'end': '04/10/1992', 'color': 'BV'}, 'AB': {'start': 'start', 'end': '01/10/1987', 'color': 'D'}, 'DC': {'start': '10/08/1990', 'end': '25/08/1990', 'color': 'D'}, 'DF': {'start': '16/02/1988', 'end': '29/05/1988', 'color': 'D'}, 'CC': {'start': '27/05/1988', 'end': '01/06/1990', 'color': 'D'}, 'JE': {'start': '05/02/1989', 'end': '19/07/1989', 'color': 'RG'}, 'DP': {'start': '20/09/1990', 'end': '22/09/1990', 'color': 'D'}, 'DG': {'start': '25/11/1990', 'end': 'end', 'color': 'BV'}, 'JD': {'start': '15/07/1993', 'end': '31/07/1993', 'color': 'T'}, 'PS': {'start': '23/09/1993', 'end': 'end', 'color': 'BV'}, 'LG': {'start': '15/07/1993', 'end': '08/01/1994', 'color': 'T'}, 'MC': {'start': '01/02/1994', 'end': 'end', 'color': 'T'}}


In [163]:
text = timeline_section
# Get ImageSize, PlotArea, Alignbars, DateFormat, Period, TimeAxis, Legend, ScaleMajor, ScaleMinor and Colors
pattern = r'(?P<key>ImageSize|PlotArea|Alignbars|DateFormat|Period|TimeAxis|Legend|ScaleMajor|ScaleMinor|Colors)\s*=\s*(?P<value>[^=\n]*)(?=\n\S|\Z)'
metadata = dict(re.findall(pattern, text))

# Get BarData
pattern = r'(?<=BarData\s*=\s*\n)(?P<value>.*?)(?=\n\n|\Z)'
bardata = {}
for line in re.findall(pattern, text, flags=re.DOTALL):
    for match in re.findall(r'(?P<bar>bar:\S+)\s+text:(?P<text>.+)', line):
        bardata[match[0]] = match[1]

# Get PlotData
pattern = r'(?<=PlotData\s*=\s*\n)(?P<value>.*?)(?=\n\n|\Z)'
plotdata = []
for line in re.findall(pattern, text, flags=re.DOTALL):
    values = re.findall(r'\w+:[^ ]+', line)
    plotdata.append({v.split(':')[0]: v.split(':')[1] for v in values})

print(metadata)
print(bardata)
print(plotdata)

error: look-behind requires fixed-width pattern

In [164]:
import re

def parse_line(line):
    key_value_pattern = r'(?P<key>[\w ]+) = (?P<value>.+)'
    match = re.match(key_value_pattern, line)
    if match:
        key = match.group('key').strip()
        value = match.group('value').strip()
        return key, value
    else:
        return None

def parse_colors(raw_colors):
    color_pattern = r'id:(?P<id>\w+) +value:(?P<value>[^ ]+) +legend:(?P<legend>[^ ]+)'
    colors = {}
    for line in raw_colors.split('\n'):
        match = re.match(color_pattern, line)
        if match:
            color_id = match.group('id')
            color_value = match.group('value')
            color_legend = match.group('legend')
            colors[color_id] = {'value': color_value, 'legend': color_legend}
    return colors

def parse_bar_data(raw_bar_data):
    bar_pattern = r'bar:(?P<bar>\w+) +text:(?P<text>.+)'
    bars = {}
    for line in raw_bar_data.split('\n'):
        match = re.match(bar_pattern, line)
        if match:
            bar_name = match.group('bar')
            bar_text = match.group('text')
            bars[bar_name] = bar_text
    return bars

def parse_plot_data(raw_plot_data):
    plot_pattern = r'width:(?P<width>\d+) *\nbar:(?P<bar>\w+) +from:(?P<from>[^ ]+) +till:(?P<till>[^ ]+) +color:(?P<color>\w+)'
    plot_data = []
    for line in raw_plot_data.split('\n'):
        match = re.match(plot_pattern, line)
        if match:
            width = match.group('width')
            bar = match.group('bar')
            start = match.group('from')
            end = match.group('till')
            color = match.group('color')
            plot_data.append({'width': width, 'bar': bar, 'start': start, 'end': end, 'color': color})
    return plot_data

def parse_timeline(raw_text):
    timeline_pattern = r'{{#tag:timeline\|(.*)}}'
    match = re.search(timeline_pattern, raw_text, flags=re.DOTALL)
    if match:
        raw_timeline = match.group(1)
        raw_lines = raw_timeline.split('\n')
        timeline = {}
        for i, line in enumerate(raw_lines):
            if line.endswith('='):
                key = line.strip()[:-1]
                timeline[key] = []
                parse_function = None
                if key == 'Colors':
                    parse_function = parse_colors
                elif key == 'BarData':
                    parse_function = parse_bar_data
                elif key == 'PlotData':
                    parse_function = parse_plot_data
                if parse_function:
                    j = i+1
                    while j < len(raw_lines) and not raw_lines[j].endswith('='):
                        timeline[key].append(parse_function(raw_lines[j]))
                        j += 1
            else:
                parsed_line = parse_line(line)
                if parsed_line:
                    key, value = parsed_line
                    if key in timeline:
                        timeline[key].append(value)
                    else:
                        timeline[key] = [value]
        return timeline
    else:
        return None

In [141]:
# split the text into lines
lines = timeline_section.strip().split('\n')
# create an empty dictionary
data = {}

In [173]:
# # iterate through the lines and add key-value pairs to the dictionary
# for line in lines:
#     # split each line into key-value pairs
#     key_value = line.strip().split(' = ')
#     if '=' in line:
#         section = line.split(' ')[-1]
#         print(section)

# # print the dictionary
# print(data)

In [150]:
lines

['ImageSize = width:960 height:auto barincrement:22',
 'PlotArea = left:90 bottom:90 top:0 right:0',
 'Alignbars = justify',
 'DateFormat = dd/mm/yyyy',
 'Period = from:01/03/1987 till:05/04/1994',
 'TimeAxis = orientation:horizontal format:yyyy',
 'Legend = orientation:vertical position:bottom columns:4',
 'ScaleMajor = increment:1 start:1988',
 '',
 'Colors =',
 '  id:V      value:red             legend:Lead_vocals',
 '  id:BV     value:pink            legend:Backing_vocals',
 '  id:LG     value:teal            legend:Lead_guitar',
 '  id:RG     value:brightgreen     legend:Rhythm_guitar',
 '  id:B      value:blue            legend:Bass',
 '  id:D      value:orange          legend:Drums',
 '  id:C      value:purple          legend:Cello',
 '  id:T      value:yellow          legend:Touring_member',
 '  id:studio value:black           legend:Studio_releases',
 '  id:other  value:gray(0.5)       legend:Other_releases',
 '  id:bars   value:gray(0.93)',
 '',
 'BackgroundColors = bars:bars

{'ImageSize': 'width:960 height:auto barincrement:22',
 'PlotArea': 'left:90 bottom:90 top:0 right:0',
 'Alignbars': 'justify',
 'DateFormat': 'dd/mm/yyyy',
 'Period': 'from:01/03/1987 till:05/04/1994',
 'TimeAxis': 'orientation:horizontal format:yyyy',
 'Legend': 'orientation:vertical position:bottom columns:4',
 'ScaleMajor': 'increment:1 start:1988',
 'BackgroundColors': 'bars:bars'}